<a href="https://colab.research.google.com/github/Beluuuuuuga/python_machine_learning_2nd_edition/blob/master/Python%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6.1　パイプラインによるワークフローの効率化

## 6.1.1　データセット読み込み

In [0]:
import pandas as pd
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data', header=None)

In [3]:
from sklearn.preprocessing import LabelEncoder
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [11]:
# locは行・列にアクセスする
X = df.loc[:, 2:].values # 2列から
y = df.loc[:, 1].values # 2列のみ
le = LabelEncoder()
y = le.fit_transform(y)
le.classes_

array(['B', 'M'], dtype=object)

In [12]:
# ラベルをエンコード
le.transform(['M', 'B'])

array([1, 0])

In [0]:
from sklearn.model_selection import  train_test_split
# テストデータと分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y, random_state=1)

## 6.1.2　パイプラインで変換器と推定器を結合する


In [16]:
from sklearn.preprocessing import  StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

# パイプラインでスケーリング、主成分分析、ロジスティック回帰を結合
# 30次元を2次元に圧縮
pipe_lr = make_pipeline(StandardScaler(), PCA(n_components=2), LogisticRegression(random_state=1))
pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_test)
print('Test Accuracy: %3f' % pipe_lr.score(X_test, y_test))

Test Accuracy: 0.956140


# 6.2　k分割交差検証を使ったモデルの性能の評価

## 6.2.1　ホールドアウト法

## 6.2.2　k分割交差検証

In [24]:
# 層化k分割交差検証
import numpy as np
from sklearn.model_selection import StratifiedKFold
# 分割数、乱数生成器の状態
kfold = StratifiedKFold(n_splits=10, random_state=1).split(X_train, y_train)
scores = []
for k, (train, test) in enumerate(kfold):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)
    print('Fold: %2d, Class dist.: %s. Acc: %.3f' % (k+1, np.bincount(y_train[train]), score) )

# 正解率の平均と標準偏差を出力
print('\nCV accuracy: %.3f  std: %.3f' % (np.mean(scores), np.std(scores)))

Fold:  1, Class dist.: [256 153]. Acc: 0.935
Fold:  2, Class dist.: [256 153]. Acc: 0.935
Fold:  3, Class dist.: [256 153]. Acc: 0.957
Fold:  4, Class dist.: [256 153]. Acc: 0.957
Fold:  5, Class dist.: [256 153]. Acc: 0.935
Fold:  6, Class dist.: [257 153]. Acc: 0.956
Fold:  7, Class dist.: [257 153]. Acc: 0.978
Fold:  8, Class dist.: [257 153]. Acc: 0.933
Fold:  9, Class dist.: [257 153]. Acc: 0.956
Fold: 10, Class dist.: [257 153]. Acc: 0.956

CV accuracy: 0.950  std: 0.014


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
